In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import pickle as pkl

In [3]:
%run utils.ipynb
%run targets.ipynb

In [4]:
# Get Data
df = pd.read_csv('data/ETHBTC_1m_1519496760000_1549568220000.csv')
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [5]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE
0  1519496760000  0.085541  0.085570  0.085407  0.085473
1  1519496820000  0.085473  0.085570  0.085448  0.085449
2  1519496880000  0.085448  0.085569  0.085400  0.085538
3  1519496940000  0.085538  0.085546  0.085402  0.085540
4  1519497000000  0.085539  0.085552  0.085464  0.085464

In [6]:
data_validation(data)

In [7]:
# Get Target
%run targets.ipynb
target = get_target_1()
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [8]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape

(499999, 6)

In [9]:
data_validation(data)

In [10]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299999, 6)

(100000, 6)

(100000, 6)

In [20]:
# Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = scaler.fit(TR_data.iloc[:, 1:-1].values)

TR_data.iloc[:, 1:-1] = scaler.transform(TR_data.iloc[:, 1:-1])
CV_data.iloc[:, 1:-1] = scaler.transform(CV_data.iloc[:, 1:-1])
TT_data.iloc[:, 1:-1] = scaler.transform(TT_data.iloc[:, 1:-1])

In [53]:
win=240
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=win, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=win, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=win, tt_win=0, point_target=True)

In [54]:
TR_X.shape
TR_y.shape

(299760, 961)

(299760,)

In [44]:
from sklearn.ensemble import RandomForestClassifier


params = {
    'n_estimators':11,
    'criterion':'entropy',
    'max_depth':None,
    'min_samples_split':2,
    'min_samples_leaf':1,
    'max_features':'sqrt',
    'max_leaf_nodes':None,
    'bootstrap':False,
    'oob_score':False,
    'n_jobs':-1,
    'random_state':0,
    'verbose':1,
    'warm_start':False,
    'class_weight':None
}

for max_features in [11]:
    print(max_features)
    params['max_features'] = 'sqrt'
    
    model = RandomForestClassifier(**params)
    model = model.fit(TR_X[:, 1:], TR_y)
    
    TR_y_pred = model.predict_proba(TR_X[:, 1:])[:, 1]
    CV_y_pred = model.predict_proba(CV_X[:, 1:])[:, 1]
    TT_y_pred = model.predict_proba(TT_X[:, 1:])[:, 1]
    
    TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
    
    res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
    res = res.append(pd.Series(TR_metrics, name='TR'))
    res = res.append(pd.Series(CV_metrics, name='CV'))
    res = res.append(pd.Series(TT_metrics, name='TT'))
    
    print(res)

11


[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed: 16.1min finished
[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    1.3s finished
[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    1.0s finished


         acc       auc        f1   log_loss
TR  0.999997  0.999997  0.999997   0.000115
CV  0.502730  0.502119  0.483776  17.175304
TT  0.502620  0.501358  0.473460  17.179097


In [46]:
TR_y_pred = model.predict(TR_X[:, 1:])
TR_y_pred[:5]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TR_metrics

[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    1.5s finished


array([0., 0., 0., 0., 0.])

{'acc': 0.999996664665466,
 'auc': 0.9999965238187936,
 'f1': 0.9999965238067098,
 'log_loss': 0.00011519837367490577}

In [47]:
(TR_y_pred < 0.5).sum()

155985

In [50]:
CV_y_pred = model.predict(CV_X[:, 1:])
CV_y_pred[:5]

CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics

[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    0.5s finished


array([1., 0., 0., 0., 0.])

{'acc': 0.5027298864968294,
 'auc': 0.5021191822263793,
 'f1': 0.4837763634094597,
 'log_loss': 17.175304391679195}

In [51]:
TT_y_pred = model.predict(TT_X[:, 1:])
TT_y_pred[:5]

TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics

[Parallel(n_jobs=4)]: Done  11 out of  11 | elapsed:    0.6s finished


array([0., 1., 1., 1., 1.])

{'acc': 0.5026196892437463,
 'auc': 0.5013579859064216,
 'f1': 0.47346038412183306,
 'log_loss': 17.17909689248263}

In [52]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL', 
    'win':win
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': 'RFC',
    'n_est':11,
    'accuracy':'{:.4f}'.format((CV_metrics['acc']+TT_metrics['acc'])/2)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('models/'+time_tag+'.pkl', 'wb')
pkl.dump(model, f, -1)
f.close()

%notebook -e models/{big_tag}.ipynb